In [1]:
import ee
import geemap

In [2]:
import ee
import collections
collections.Callable = collections.abc.Callable

In [3]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWgwGFhm7XRS0IJdlDRiOMQ5xZELEFASw8Zpy_mTsyS2sTnqIGMav3Q



Successfully saved authorization token.


In [4]:
ee.Initialize()

## Imports

In [5]:
import ee
import geemap

Map = geemap.Map()

geometry = ee.Geometry.Polygon(
        [[[25.67758947139908, -10.52500539578282],
          [25.67758947139908, -10.831343855184292],
          [26.285562854675604, -10.831343855184292],
          [26.285562854675604, -10.52500539578282]]], None, False)
mines = ee.FeatureCollection("users/EmilyNason/cod_mines_curated_all_opendata_p_ipis")
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
gsw = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")
area3 = ee.Geometry.Polygon(
        [[[26.117306035200816, -10.551319560770521],
          [26.117306035200816, -10.632650522900946],
          [26.248455327193003, -10.632650522900946],
          [26.248455327193003, -10.551319560770521]]], None, False)
area1 = ee.Geometry.Polygon(
        [[[25.783982219615126, -10.760592214819402],
          [25.783982219615126, -10.804773403670321],
          [25.882344188609267, -10.804773403670321],
          [25.882344188609267, -10.760592214819402]]], None, False)
area2 = ee.Geometry.Polygon(
        [[[25.883463192760367, -10.653415341387422],
          [25.883463192760367, -10.684961077984651],
          [25.918653775035757, -10.684961077984651],
          [25.918653775035757, -10.653415341387422]]], None, False)

## True Color Composites

In [6]:
Map.centerObject(geometry, 11)

# Applies scaling factors.
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True) \
              .addBands(thermalBands, None, True)

before = l8.filter(ee.Filter.lt('CLOUD_COVER', 30)) \
  .filter(ee.Filter.date('2016-06-01', '2017-01-01')) \
  .filter(ee.Filter.intersects('.geo', geometry))
after = l8.filter(ee.Filter.lt('CLOUD_COVER', 30)) \
  .filter(ee.Filter.date('2021-06-01', '2022-01-01')) \
  .filter(ee.Filter.intersects('.geo', geometry))
before = before.map(applyScaleFactors).median().clip(geometry)
after = after.map(applyScaleFactors).median().clip(geometry)

rgbVis = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0.0,
  'max': 0.3,
}

Map.addLayer(before, rgbVis, '2016')
Map.addLayer(after, rgbVis, '2021')

## Indices

In [7]:
def addIndices(image):
  ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename(['ndvi']); # NIR, RED
  clay = image.normalizedDifference(['SR_B6', 'SR_B7']).rename(['clay']); # SWIR1, SWIR2
  iron = image.normalizedDifference(['SR_B4', 'SR_B2']).rename(['iron']); # RED, BLUE
  ndmi = image.normalizedDifference(['SR_B5', 'SR_B6']).rename(['ndmi']); # NIR, SWIR1
  # add in VII & WDI
  vigs = image.expression(
    '((A-B)/(A+B)) + ((C-B)/(C+B))/2 + ((C-D)/(C+D)) + ((C-D)/(C+D))/2 + ((C-E)/(C+E)) + ((C-E)/(C+E))/2', {
    'A': image.select('SR_B3'), # GREEN
    'B': image.select('SR_B4'), # RED
    'C': image.select('SR_B5'), # NIR
    'D': image.select('SR_B6'), # SWIR1
    'E': image.select('SR_B7'), # SWIR2
    }).rename('vigs')
  return image.addBands(ndvi).addBands(clay).addBands(iron).addBands(ndmi).addBands(vigs)

afterIndices = addIndices(after)
beforeIndices = addIndices(before)
ndviVis = {'min':0, 'max':1, 'palette': ['white', 'green']}
clayVis = {'min':0, 'max':0.5, 'palette': ['white', 'brown']}
ironVis = {'min':0, 'max':0.5, 'palette': ['white', 'gray']}
ndmiVis = {'min':0, 'max':0.5, 'palette': ['white', 'blue']}
vigsVis = {'min':0, 'max':1, 'palette': ['white', 'purple']}

Map.addLayer(beforeIndices.select('ndvi'), ndviVis, 'NDVI 2016')
Map.addLayer(afterIndices.select('ndvi'), ndviVis, 'NDVI 2021')
Map.addLayer(beforeIndices.select('clay'), clayVis, 'Clay 2016')
Map.addLayer(afterIndices.select('clay'), clayVis, 'Clay 2021')
Map.addLayer(beforeIndices.select('iron'), ironVis, 'Iron 2016')
Map.addLayer(afterIndices.select('iron'), ironVis, 'Iron 2021')
# Map.addLayer(beforeIndices.select('ndmi'), ndmiVis, 'NDMI 2016', False)
# Map.addLayer(afterIndices.select('ndmi'), ndmiVis, 'NDMI 2021', False)
Map.addLayer(beforeIndices.select('vigs'), vigsVis, 'VIGS 2016')
Map.addLayer(afterIndices.select('vigs'), vigsVis, 'VIGS 2021')

## Calculated Changes in Indices

In [8]:
# Changes in Indices
cpThreshold = 8
permanentWater = gsw.select('seasonality').gte(5).clip(geometry)

# NDVI
# low ndvi -> possible existing mining
# 2016
miningNDVI2016 = beforeIndices.select('ndvi').lt(0.15).rename('miningNDVI').selfMask()
miningNDVI2016 = miningNDVI2016.where(permanentWater,0).selfMask()
connectNDVI2016 = miningNDVI2016.connectedPixelCount(25)
miningNDVI2016 = miningNDVI2016.updateMask(connectNDVI2016.gt(cpThreshold))
Map.addLayer(miningNDVI2016, {'min':0, 'max':1, 'palette': ['yellow']}, 'Possible Mining Area 2016')
# 2021
miningNDVI2021 = afterIndices.select('ndvi').lt(0.15).rename('miningNDVI').selfMask()
miningNDVI2021 = miningNDVI2021.where(permanentWater,0).selfMask()
connectNDVI2021 = miningNDVI2021.connectedPixelCount(25)
miningNDVI2021 = miningNDVI2021.updateMask(connectNDVI2021.gt(cpThreshold))
Map.addLayer(miningNDVI2021, {'min':0, 'max':1, 'palette': ['orange']}, 'Possible Mining Area 2021')
#ndvi loss -> possible new mining
differenceNDVI = beforeIndices.select('ndvi').divide(afterIndices.select('ndvi'))
threshold = 1.75
newMiningNDVI = differenceNDVI.gt(threshold).rename('newMiningNDVI').selfMask()
newMiningNDVI = newMiningNDVI.where(permanentWater,0).selfMask()
connectNDVI = newMiningNDVI.connectedPixelCount(25)
newMiningNDVI = newMiningNDVI.updateMask(connectNDVI.gt(cpThreshold))
Map.addLayer(newMiningNDVI, {'min':0, 'max':1, 'palette': ['red']}, 'Possible New Mining Area')

# VIGS loss -> becomes an outline of new mining sites
# shows new strained vegetation - demonstration of how expanded mining has expanded vegetation stress
# could identify mines through analysis of areas surrounded by new vegetation stress
differenceVIGS = beforeIndices.select('vigs').divide(afterIndices.select('vigs'))
threshold = 1.75
newMiningVIGS = differenceVIGS.gt(threshold).rename('newMining').selfMask()
newMiningVIGS = newMiningVIGS.where(permanentWater,0).selfMask()
connectVIGS = newMiningVIGS.connectedPixelCount(25)
newMiningVIGS = newMiningVIGS.updateMask(connectVIGS.gt(cpThreshold))
Map.addLayer(newMiningVIGS, {'min':0, 'max':1, 'palette': ['D772FD']}, 'New Vegetation Stress Due to Mining (VIGS)')

# CLAY
# low clay - potential mining -> clay area possibly has been cleared
# 2016
miningClay2016 = beforeIndices.select('clay').lt(0.08).rename('miningClay').selfMask()
miningClay2016 = miningClay2016.where(permanentWater,0).selfMask()
connectClay2016 = miningClay2016.connectedPixelCount(25)
miningClay2016 = miningClay2016.updateMask(connectClay2016.gt(cpThreshold))
Map.addLayer(miningClay2016, {'min':0, 'max':1, 'palette': ['FB9FF9']}, 'Possible Mining Area 2016 (Clay)')
# 2021
miningClay2021 = afterIndices.select('clay').lt(0.08).rename('miningClay').selfMask()
miningClay2021 = miningClay2021.where(permanentWater,0).selfMask()
connectClay2021 = miningClay2021.connectedPixelCount(25)
miningClay2021 = miningClay2021.updateMask(connectClay2021.gt(cpThreshold))
Map.addLayer(miningClay2021, {'min':0, 'max':1, 'palette': ['FF70FC']}, 'Possible Mining Area 2021 (Clay)')
# clay loss -> active mining
differenceClay = beforeIndices.select('clay').divide(afterIndices.select('clay'))
threshold = 1.85
newMiningClay = differenceClay.gt(threshold).rename('newMiningClay').selfMask()
newMiningClay = newMiningClay.where(permanentWater,0).selfMask()
connectClay = newMiningClay.connectedPixelCount(25)
newMiningClay = newMiningClay.updateMask(connectClay.gt(cpThreshold))
Map.addLayer(newMiningClay, {'min':0, 'max':1, 'palette': ['DA09D6']}, 'Possible New Mining Area (loss of Clay)')

# IRON
# high iron - looks like it is picking up infrastructure
# 2016
miningIron2016 = beforeIndices.select('iron').gt(0.4).rename('miningIron').selfMask()
miningIron2016 = miningIron2016.where(permanentWater,0).selfMask()
connectIron2016 = miningIron2016.connectedPixelCount(25)
miningIron2016 = miningIron2016.updateMask(connectIron2016.gt(cpThreshold))
Map.addLayer(miningIron2016, {'min':0, 'max':1, 'palette': ['A9D3FE']}, 'Possible Infrastructure 2016 (Iron)')
# 2021
miningIron2021 = afterIndices.select('iron').gt(0.4).rename('miningIron').selfMask()
miningIron2021 = miningIron2021.where(permanentWater,0).selfMask()
connectIron2021 = miningIron2021.connectedPixelCount(25)
miningIron2021 = miningIron2021.updateMask(connectIron2021.gt(cpThreshold))
Map.addLayer(miningIron2021, {'min':0, 'max':1, 'palette': ['2F94F9']}, 'Possible Infrastructure 2021 (Iron)')
# iron gain - potential mining activies increase -> more infrastructure built
differenceIron = afterIndices.select('iron').divide(beforeIndices.select('iron'))
threshold = 1.3; # trial and error tried a few different thresholds
newMiningIron = differenceIron.gt(threshold).rename('newMiningIron').selfMask()
newMiningIron = newMiningIron.where(permanentWater,0).selfMask()
connectIron = newMiningIron.connectedPixelCount(25)
newMiningIron = newMiningIron.updateMask(connectIron.gt(cpThreshold))
Map.addLayer(newMiningIron, {'min':0, 'max':1, 'palette': ['2004E7']}, 'Possible New Mining Infrastructure (Iron)')

# LEGEND
legend_dict = {
    'NDVI Level': ('008000'), # green
    'Low NDVI 2016': ('FFFF00'), # yellow
    'Low NDVI 2021': ('FFA500'), # orange
    'NDVI Lost': ('FF0000'), # red
    'VIGS Level': ('7F00FF'), # violet
    'New Vegetation Stress (VIGS)': ('D772FD'), # light purple
    'Clay level': ('964B00'), # brown
    'Low Clay 2016': ('FB9FF9'), # light pink
    'Low Clay 2021': ('FF70FC'), # medium pink
    'Clay Lost': ('DA09D6'), # dark pink
    'Iron level': ('808080'), #gray
    'Iron 2016': ('A9D3FE'), # light blue
    'Iron 2021': ('2F94F9'), # medium blue
    'Iron Gained': ('2004E7'), # dark blue
}
# Map.remove_legend()
Map.add_legend(legend_title='Legend', legend_dict=legend_dict, position='bottomleft')
Map
# may need to open map to full screen to access all layers

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

## Mine Estimations

In [9]:
Map2 = geemap.Map()
Map2.centerObject(geometry, 11)
Map2.addLayer(before, rgbVis, '2016')
Map2.addLayer(after, rgbVis, '2021')

# combine for mine area estimate - low NDVI, low Clay, high Iron
# 2016
miningNDVI2016 = miningNDVI2016.select('miningNDVI').rename('mining')
miningClay2016 = miningClay2016.select('miningClay').rename('mining')
miningIron2016 = miningIron2016.select('miningIron').rename('mining')
pMines2016 = ee.ImageCollection([miningNDVI2016, miningClay2016, miningIron2016]).mosaic()
Map2.addLayer(pMines2016, {'min':0, 'max':1, 'palette': ['orange']}, 'Initial Mine Estimate 2016')
mineSize2016 = pMines2016.select('mining') \
  .connectedPixelCount(**{
    'maxSize': 128, 'eightConnected': True
  })
pixelArea = ee.Image.pixelArea()
mineArea2016 = mineSize2016.multiply(pixelArea)
areaMask2016 = mineArea2016.gte(200000); #m^2
pMines2016Final = pMines2016.updateMask(areaMask2016)
Map2.addLayer(pMines2016Final, {'min':0, 'max':1, 'palette': ['red']}, 'Mine Estimate 2016')

# 2021
miningNDVI2021 = miningNDVI2021.select('miningNDVI').rename('mining')
miningClay2021 = miningClay2021.select('miningClay').rename('mining')
miningIron2021 = miningIron2021.select('miningIron').rename('mining')
pMines2021 = ee.ImageCollection([miningNDVI2021, miningClay2021, miningIron2021]).mosaic()
Map2.addLayer(pMines2021, {'min':0, 'max':1, 'palette': ['green']}, 'Initial Mine Estimate 2021')
mineSize2021 = pMines2021.select('mining') \
  .connectedPixelCount(**{
    'maxSize': 128, 'eightConnected': True
  })
mineArea2021 = mineSize2021.multiply(pixelArea)
areaMask2021 = mineArea2021.gte(200000); #m^2
pMines2021Final = pMines2021.updateMask(areaMask2021)
Map2.addLayer(pMines2021Final, {'min':0, 'max':1, 'palette': ['blue']}, 'Mine Estimate 2021')

Map2.addLayer(mines, {'color': 'E6EAEE'}, 'mines'); # light gray

# LEGEND
legend_dict = {
    'Initial Mine Estimate 2016': ('FFA500'), # orange
    'Final Mine Estimate 2016': ('FF0000'), # red
    'Initial Mine Estimate 2021': ('008000'), # green
    'Final Mine Estimate 2021': ('0000FF'), # blue
    'Known Active Mines': ('E6EAEE'), # light gray
}
Map2.add_legend(legend_title='Legend', legend_dict=legend_dict, position='bottomright')
Map2

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…